In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import kagglehub
import sqlite3
import plotly.express as px



In [2]:
fpath=kagglehub.dataset_download("rohanrao/formula-1-world-championship-1950-2020")
circuits = pd.read_csv(f'{fpath}/circuits.csv', index_col=0, na_values=r'\N')
constructorResults = pd.read_csv(f'{fpath}/constructor_results.csv', index_col=0, na_values=r'\N')
constructors = pd.read_csv(f'{fpath}/constructors.csv', index_col=0, na_values=r'\N')
constructorStandings = pd.read_csv(f'{fpath}/constructor_standings.csv', index_col=0, na_values=r'\N')
drivers = pd.read_csv(f'{fpath}/drivers.csv', index_col=0, na_values=r'\N')
driverStandings = pd.read_csv(f'{fpath}/driver_standings.csv', index_col=0, na_values=r'\N')
lapTimes = pd.read_csv(f'{fpath}/lap_times.csv')
pitStops = pd.read_csv(f'{fpath}/pit_stops.csv')
qualifying = pd.read_csv(f'{fpath}/qualifying.csv', index_col=0, na_values=r'\N')
races = pd.read_csv(f'{fpath}/races.csv', na_values=r'\N')
results = pd.read_csv(f'{fpath}/results.csv', index_col=0, na_values=r'\N')
seasons = pd.read_csv(f'{fpath}/seasons.csv', index_col=0, na_values=r'\N')
status = pd.read_csv(f'{fpath}/status.csv', index_col=0, na_values=r'\N')

In [3]:
try:
    # Establish an SQLite database connection (in-memory or persistent)
    conn = sqlite3.connect(':memory:')  # Use ':memory:' for an in-memory database
    # Alternatively, use a file-based database:
    # conn = sqlite3.connect('f1_data.db')

    # Load all CSV files into SQLite
    circuits.to_sql('circuits', conn, index=True, if_exists='replace')
    constructorResults.to_sql('constructor_results', conn, index=False, if_exists='replace')
    constructors.to_sql('constructors', conn, index=True, if_exists='replace')
    constructorStandings.to_sql('constructor_standings', conn, index=False, if_exists='replace')
    drivers.to_sql('drivers', conn, index=True, if_exists='replace')
    driverStandings.to_sql('driver_standings', conn, index=False, if_exists='replace')
    lapTimes.to_sql('lap_times', conn, index=False, if_exists='replace')
    pitStops.to_sql('pit_stops', conn, index=False, if_exists='replace')
    qualifying.to_sql('qualifying', conn, index=False, if_exists='replace')
    races.to_sql('races', conn, index=True, if_exists='replace')
    results.to_sql('results', conn, index=True, if_exists='replace')
    seasons.to_sql('seasons', conn, index=False, if_exists='replace')
    status.to_sql('status', conn, index=False, if_exists='replace')

    print("All tables have been successfully loaded into SQLite!")

except sqlite3.Error as e:
    print(f"SQLite error: {e}")
except Exception as e:
    print(f"General error: {e}")

All tables have been successfully loaded into SQLite!


In [4]:
# Finding Sergio Perez
query = "SELECT driverId, driverRef, code, forename, surname FROM drivers WHERE surname='Pérez'"
findPerez = pd.read_sql(query, conn)
findPerez

,driverId,driverRef,code,forename,surname
0,815,perez,PER,Sergio,Pérez


In [5]:
# Find Red Bull constructorId
query= "SELECT * FROM constructors WHERE name='Red Bull'"
findConstructor =  pd.read_sql(query, conn)
findConstructor


,constructorId,constructorRef,name,nationality,url
0,9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing


In [6]:

# Find all Sergios races in Red Bull
query="SELECT raceId, driverId, constructorId, grid, position, points FROM results WHERE driverId = '815' AND constructorId = '9' ORDER BY raceId"
lapTimes_df = pd.read_sql(query, conn)
lapTimes = lapTimes_df.to_sql('perezLapTimes', conn, index=False, if_exists='replace')
# Adding year
query="SELECT perezLapTimes.*, races.year FROM perezLapTimes INNER JOIN races ON perezLapTimes.raceId = races.raceId"
lapTimes_df = pd.read_sql(query, conn)
lapTimes = lapTimes_df.to_sql('perezLapTimes', conn, index=False, if_exists='replace')
lapTimes_df

,raceId,driverId,constructorId,grid,position,points,year
0,1051,815,9,11,4.0,12.0,2021
1,1052,815,9,0,5.0,10.0,2021
2,1053,815,9,2,11.0,0.0,2021
3,1054,815,9,4,4.0,12.0,2021
4,1055,815,9,8,5.0,10.0,2021
...,...,...,...,...,...,...,...
73,1128,815,9,16,NaN,0.0,2024
74,1129,815,9,16,NaN,0.0,2024
75,1130,815,9,11,8.0,4.0,2024
76,1131,815,9,8,7.0,6.0,2024


In [7]:
# Ensure 'points' is numeric and handle any potential issues
lapTimes_df['points'] = pd.to_numeric(lapTimes_df['points'], errors='coerce')

# Prepare data for plotting
driver_points_by_race = lapTimes_df.rename(columns={'raceId': 'Race ID', 'points': 'Points', 'year': 'Year'})

# Create line chart
fig = px.line(
    driver_points_by_race,
    x='Race ID',
    y='Points',
    title='Points Scored per Race',
    labels={'Points': 'Points Scored', 'Race ID': 'Race ID'},
)

# Find the first Race ID of each year
first_race_of_year = driver_points_by_race.groupby('Year').first().reset_index()

# Add vertical lines to mark the beginning of each year on the x-axis
fig.update_layout(
    shapes=[
        dict(
            type='line',
            x0=row['Race ID'],
            x1=row['Race ID'],
            y0=0,
            y1=driver_points_by_race['Points'].max(),
            line=dict(color='red', width=2, dash='dash'),
        )
        for _, row in first_race_of_year.iterrows()
    ],
    annotations=[
        dict(
            x=row['Race ID'],
            y=27,  # Position the label slightly below the x-axis
            text=str(row['Year']),
            showarrow=False,
            font=dict(size=12, color='black'),
            align='center',
        )
        for _, row in first_race_of_year.iterrows()
    ],
    xaxis_title='Race ID',
    yaxis_title='Points Scored',
)

# Show the plot
fig.show()

In [18]:
query = "SELECT raceId, position FROM qualifying WHERE driverId = '815' and constructorId = '9' ORDER BY raceId"
quali_df = pd.read_sql(query, conn)
quali = quali_df.to_sql('quali', conn, index=False, if_exists='replace')
query = "SELECT quali.*, perezLapTimes.year FROM quali INNER JOIN perezLapTimes ON quali.raceId = perezLapTimes.raceId"
#query = "SELECT * FROM perezLapTimes"
quali_df = pd.read_sql(query, conn)
quali_df




False


In [25]:

quali_df = quali_df.rename(columns={'raceId': 'Race ID', 'position': 'Position', 'year': 'Year'})

first_race_of_year = quali_df.groupby('Year').first().reset_index()
    
driver_points_by_race['Race ID'] = pd.to_numeric(driver_points_by_race['Race ID'], errors='coerce')
first_race_of_year['Race ID'] = pd.to_numeric(first_race_of_year['Race ID'], errors='coerce')


# Create line chart
fig2 = px.line(
    driver_points_by_race,
    x='Race ID',
    y='position',
    title='Qualifying position',
    labels={'Position': 'Position', 'Race ID': 'Race ID'},
)


# Update layout with shapes and annotations
fig2.update_layout(
    shapes=[
        dict(
            type='line',
            x0=row['Race ID'],  # Access 'Race ID' directly
            x1=row['Race ID'],
            y0=0,
            y1=quali_df['Position'].max(),  
            line=dict(color='red', width=2, dash='dash'),

        )
        for _, row in first_race_of_year.iterrows()
    ],
    annotations=[
        dict(
            x=row['Race ID'],  # Access 'Race ID' directly
            y=22,  # Adjust y-coordinate for better placement
            text=str(row['Year']),
            showarrow=False,
            font=dict(size=12, color='black'),
            align='center',
        )
        for _, row in first_race_of_year.iterrows()
    ],
)
fig2.update_layout(xaxis=dict(range=[driver_points_by_race['Race ID'].min(), driver_points_by_race['Race ID'].max()]))

